In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 500)

import seaborn as sns
import nltk
nltk.download('words')
pd.set_option('max_colwidth', 40)
import os
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
sws = set(stopwords.words('english'))
punctuation = set(string.punctuation)
from  helper_functions import user_event_count,\
                                ticket_types,\
                                previous_payouts,\
                                user_df,\
                                unique_col_indicator,\
                                create_target,\
                                convert_date,\
                                low_cor_cols,\
                                drop_cols,\
                                description_cols,\
                                ticket_types


[nltk_data] Downloading package words to /Users/junswj/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/junswj/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def user_event_count():
    df['user_event_count'] = df.groupby('user_created').count().max(axis=1)
    return df


def previous_payouts(df):
    new_df=pd.DataFrame(df.previous_payouts[0])
    for i in range(1,df.previous_payouts.shape[0]):
        df_element=pd.DataFrame(df.previous_payouts[i])
        new_df=pd.concat([new_df, df_element])
    return new_df

def user_df():
    unique_cols = []
    for col in columns:
        if unique_col_indicator(df,col):
            unique_cols.append(col)
    user_unique_events = df.groupby('user_created').count().max(axis=1)
    user_unique_events = pd.DataFrame(user_unique_events,columns=['user_event_count'])
    return df[unique_cols].merge(user_unique_events,how='inner',on='user_created')

def unique_col_indicator(df,col):
    try:
        col_count = df.groupby(['user_created',col])['channels'].count().shape[0]
        user_count = df.groupby(['user_created'])['channels'].count().shape[0]
        if col_count == user_count:
            return True
        else:

            return False
    except:
        pass

    
#check
def user_unique_events_col():
    user_unique_events = df.groupby('user_created').count().max(axis=1)
    user_unique_events =  pd.DataFrame(user_unique_events,columns=['user_event_count'])
    return df.merge(user_unique_events,how='inner',on='user_created')

#check
# def create_target():
#     df['fraud'] = df['acct_type'].map({'fraudster_event': 1,
#                                    'premium': 0,
#                                    'spammer_warn': 0,
#                                    'fraudster': 1,
#                                    'spammer_limited': 0,
#                                    'spammer_noinvite': 0,
#                                    'locked': 0,  #changed to 0 from 1
#                                    'tos_lock': 0,
#                                    'tos_warn': 0,
#                                    'fraudster_att': 1,
#                                    'spammer_web': 0,
#                                    'spammer': 0})
#     return df

from datetime import datetime

#check
def convert_date():
    df["approx_payout_date"] = pd.to_datetime(df["approx_payout_date"], unit = 's')
    df["event_created"] = pd.to_datetime(df["event_created"], unit = 's')
    df["event_end"] = pd.to_datetime(df["event_end"], unit = 's')
    df["event_start"] = pd.to_datetime(df["event_start"], unit = 's')
    df["event_published"] = pd.to_datetime(df["event_published"], unit = 's')
    return df

def low_cor_cols(r_score):
    df[num_cols()].corr()[['fraud']].values
    corr_df = df[num_cols()].corr()[['fraud']].sort_values('fraud')
    low_corr_mask = corr_df.sort_values('fraud').abs().lt(r_score).values.reshape(1,-1)[0]
    return corr_df[low_corr_mask]

def get_corrs():
    corr_df = df[num_cols()].corr()[['fraud']].abs().sort_values('fraud')
    return corr_df

def num_cols():
    return df.describe().columns

def drop_cols():
    df1 = df.drop(low_cor_cols(.09).index.tolist(),axis=1)
    df2 = df1.drop(['acct_type','sale_duration','fraud','event_start','approx_payout_date',
                    'event_published','event_end','venue_state',
                    'ticket_types','venue_name','venue_state','venue_country','venue_address',
                    'org_desc','org_name','previous_payouts','email_domain','name','currency','country','event_created'],axis=1)
    return df2

#check
def description_cols():
    import clean_desc
    df['description'] = df['description'].apply(strip_tags)
    clean_desc(df)
    return df

#check
def ticket_types(df):
    new_df=pd.DataFrame(df.ticket_types[0])
    result_df=new_df[['quantity_sold', 'quantity_total', 'event_id']].groupby('event_id').sum()
    result_df=result_df.join(new_df[['availability', 'cost', 'event_id']].groupby('event_id').mean())

    for i in range(1,df.shape[0]):
        try:
            df_element=pd.DataFrame(df.ticket_types[i])
            grp_ele_df=df_element[['quantity_sold', 'quantity_total', 'event_id']].groupby('event_id').sum()
            grp_ele_df=grp_ele_df.join(df_element[['availability', 'cost', 'event_id']].groupby('event_id').mean())
            result_df=pd.concat([result_df, grp_ele_df])

        except:
            continue
    result_df.reset_index(inplace=True)
    result_df.columns=['object_id', 'quantity_sold', 'quantity_total', 'availability', 'cost']
    final_df=df.set_index('object_id').join(result_df.set_index('object_id'))
    final_df.reset_index(inplace=True)
    final_df[['quantity_sold', 'quantity_total','availability', 'cost']]=final_df[['quantity_sold', 'quantity_total','availability', 'cost']].fillna(0)
    return final_df

def timedelta(field):
    return field.days

def event_times():
    df['publish_time_to_start'] =  df['event_published'] - df['event_created'] 
    df['publish_time_to_start'] = df['publish_time_to_start'].apply(timedelta)
    return df

def has_venue_data():
    df['has_state'] = (df['venue_state'].fillna('None').replace('','None') == 'None').astype(int)
    df['has_country'] = (df['venue_country'].fillna('None').replace('','None') == 'None').astype(int)
    df['has_address'] = (df['venue_address'].fillna('None').replace('','None') == 'None').astype(int)
    df['has_venue_name'] = (df['venue_name'].fillna('None').replace('','None') == 'None').astype(int)
    
    df['has_org_desc'] = (df['org_desc'].fillna('None').replace('','None') == 'None').astype(int)
    df['has_org_name'] = (df['org_name'].fillna('None').replace('','None') == 'None').astype(int)
    df['has_venue_name'] = (df['venue_name'].fillna('None').replace('','None') == 'None').astype(int)
    df['has_venue_name'] = (df['venue_name'].fillna('None').replace('','None') == 'None').astype(int)
    return df

def payout_type():
    df1 = pd.get_dummies(df,columns = ['payout_type','listed'])
    return df1

In [3]:
def prev_pay_count(df):
    df.previous_payouts=df.previous_payouts.apply(lambda x: len(x))
    return df

In [4]:
def col_classify(df, col, threshold):
    col_filter=(df[df.fraud==1][col].value_counts()/df[col].value_counts()).sort_values()>threshold
    col_filter_list=col_filter.where(col_filter==True).dropna().index
    df[col]=df[col].apply(lambda x : x in col_filter_list)
    df[col].astype('int64')
    return df

In [5]:
def clean_rest(df):
    df.venue_country=df.venue_country.isna()
    df.name_length=df.name_length==0
    df=pd.get_dummies(df,columns = ['payout_type','listed'])
    df=df[['body_length', 'channels', 'country', 'delivery_method', 'email_domain', 
       'fb_published', 'gts', 'has_analytics', 'has_header', 'has_logo',
        'name_length', 'num_order', 'num_payouts', 'org_desc','org_facebook',
       'org_name', 'org_twitter', 'payee_name','previous_payouts', 'show_map',
        'user_age', 'user_type',
       'venue_address', 'venue_country','quantity_sold', 'quantity_total',
       'availability', 'cost', 'fraud', 'payout_type_', 'payout_type_ACH',
       'payout_type_CHECK', 'listed_n', 'listed_y']]
    df=df.fillna(0)*1
    
    return df

In [6]:
def isexist(df, col):
    df[col]=df[col].apply(lambda x: len(x))==0
    return df

In [7]:
def country_classify(df, col='country'):
    country_filter=['JM', 'BG', 'ID', 'NG', 'PK', 'VN', 'GH', 'TR', 'SI', 'RU', 'A1', 'CH',
       'CI', 'CM', 'PS', 'JE', 'CN', 'PH', 'CZ', 'DK', 'DZ', 'NA', 'MY', 'MA',
       'KH', 'CO', 'IL']
    df[col]=df[col].apply(lambda x : x in country_filter)
    df[col].astype('int64')
    return df

In [8]:
def email_classify(df, col='email_domain'):
    email_filter=['outlook.com', 'mail.com', 'ymail.com', 'techie.com', 'inbox.com',
       'gmx.com', 'hotmail.fr', 'yahoo.fr', 'rocketmail.com',
       'mohmal.com', 'noiphuongxa.com', 'live.FR', 'nbuux.com',
       'naworld-x.com', 'myway.com', 'live.fr', 'medicalrepinsight.com',
       'lmtexformula.com', 'london.com', 'lushsaturdays.co.uk',
       'maroclancers.com', 'monkeyadvert.com', 'live.de', '126.com',
       'post.com', 'outlook.fr', 'yahoo.it', 'yahoo.de', 'yahoo.com.vn',
       'yahoo.co.id', 'vncall.net', 'visichathosting.net', 'usa.com',
       'ultimatewine.co.uk', 'twcny.rr.com', 'toke.com',
       'thinktankconsultancy.com', 'student.framingham.edu',
       'startupmaroc.com', 'socialworker.net', 'safe-mail.net',
       'rock.com', 'ravemail.com', 'quaychicago.com',
       'qualityservice.com', 'qip.ru', 'primehire.co.uk', 'lidf.co.uk',
       'photographer.net', 'petlover.com', 'ovidcapita.com', 'outlook.de',
       'levyresourcesinc.com', 'investocorp.com', 'keromail.com',
       'cyberservices.com', 'contractor.net', 'consultant.com',
       'clothmode.com', 'clerk.com', 'chef.net', 'checker.vn',
       'cdrenterprise.net', 'catchatt.com', 'cannapro.com',
       'brew-meister.com', 'brew-master.com', 'discofan.com',
       'blader.com', 'aol.co.uk', 'angelwish.org', 'anasconcept.com',
       'accountant.com', 'The2Half.com', 'Safe-mail.net',
       'DionJordan.com', '9and1.biz', '4u2nv-ent.com', '4asdkids.com',
       '31and7.com', '19sieunhan.com', 'ashfordradtech.org',
       'diversity-church.com', 'dr.com', 'emgay.com', 'kbzaverigroup.com',
       'jobsfc.com', 'jcclain.com', 'izzane.net', 'ioccupied.net',
       'yopmail.com', 'insuranceadjustersinc.com', 'instructor.net',
       'inorbit.com', 'innovateyours.com', 'indiabestplace.com',
       'indglobal-consulting.com', 'in.com', 'hushmail.com', 'hotmail.de',
       'hotelvenizbaguio.com', 'hmshost.com', 'hamptonmedi.com',
       'greenrcs.com', 'gosimplysocial.com', 'gcase.org', 'gawab.com',
       'fridayzonmarz.co.uk', 'freya.pw', 'execs.com', 'europe.com',
       'eng.uk.com', 'leisurelodgebaguio.com', 'zumba-perth.com']
    df[col]=df[col].apply(lambda x : x in email_filter)
    df[col].astype('int64')
    return df

In [9]:
def create_target(df):
    df['fraud'] = df['acct_type'].map({'fraudster_event': 1,
                                   'premium': 0,
                                   'spammer_warn': 0,
                                   'fraudster': 1,
                                   'spammer_limited': 0,
                                   'spammer_noinvite': 0,
                                   'locked': 0,  #changed to 0 from 1
                                   'tos_lock': 0,
                                   'tos_warn': 0,
                                   'fraudster_att': 1,
                                   'spammer_web': 0,
                                   'spammer': 0})
    return df

In [10]:
def pre_process_data(df):
    df=ticket_types(raw_df)
    df=create_target(df)
    df=prev_pay_count(df)
    df=email_classify(df)
    df=country_classify(df)
    df=isexist(df,'org_name')
    df=isexist(df,'payee_name')
    df=isexist(df,'venue_address')
    df=isexist(df,'org_desc')
    df=clean_rest(df)
    return df

In [11]:
raw_df = pd.read_json('data.json').drop('description',axis=1)

In [12]:
df=pre_process_data(raw_df)

In [13]:
df.head()

,body_length,channels,country,delivery_method,email_domain,fb_published,gts,has_analytics,has_header,has_logo,name_length,num_order,num_payouts,org_desc,org_facebook,org_name,org_twitter,payee_name,previous_payouts,show_map,user_age,user_type,venue_address,venue_country,quantity_sold,quantity_total,availability,cost,fraud,payout_type_,payout_type_ACH,payout_type_CHECK,listed_n,listed_y
0,3852,5,0,0.0,0,0,0.00,0,1.0,0,0,0,0,1,0.0,0,0.0,1,0,1,36,1,0,0,0.0,920.0,1.0,208.333333,1,1,0,0,0,1
1,3499,0,0,1.0,0,0,868.02,0,0.0,1,0,23,1,0,0.0,0,12.0,0,49,0,149,3,1,0,25.0,100.0,1.0,35.000000,0,0,0,1,1,0
2,2601,8,0,1.0,0,0,3500.00,0,0.0,0,0,19,4,0,0.0,0,0.0,0,36,0,214,3,0,0,48.0,48.0,1.0,93.510000,0,0,0,1,0,1
3,12347,6,0,1.0,0,0,1167.35,0,0.0,1,0,39,16,1,0.0,1,0.0,1,21,0,889,3,1,1,58.0,30000.0,1.0,13.666667,0,0,1,0,0,1
4,2417,11,0,0.0,0,1,2313.15,0,0.0,0,0,30,0,0,0.0,0,0.0,0,49,0,35,3,0,0,39.0,264.0,1.0,101.750000,0,0,0,1,0,1


In [14]:
from sklearn.model_selection import train_test_split, ShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, classification_report
from sklearn.model_selection import cross_validate

In [15]:
def X_y_split(df):
    y=df.pop('fraud')
    X=df
    return X,y

In [16]:
def tt_split(X, y, test_size=.2):
    X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=test_size)
    return X_train, X_test, y_train, y_test

In [17]:
X, y = X_y_split(df)

In [18]:
X_train, X_test, y_train, y_test= tt_split(X, y)

In [19]:
rfc = RandomForestClassifier(n_estimators=100, max_depth=2,random_state=0)

In [20]:
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=2, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [21]:
print("Mean accuracy of test: {:2.2f}".format(rfc.score(X_test, y_test)))

Mean accuracy of test: 0.97


In [22]:
y_pred=rfc.predict(X_test)

In [23]:
print(classification_report(y_test, y_pred, labels=[0,1]))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      2617
           1       0.95      0.76      0.84       251

    accuracy                           0.97      2868
   macro avg       0.96      0.88      0.91      2868
weighted avg       0.97      0.97      0.97      2868



In [24]:
#df.to_pickle('RF_data.pickle')